In [ ]:
import os
from time import time
import numpy as np
import sys
sys.path.append('../')

from utils.codification_cnn import FitnessCNN, FitnessCNNParallel, NNLayer
from utils.codification_cell import CNNLayer_WOMP, ChromosomeFilterGrow
from utils.datamanager import DataManager
from utils.codification_skipc import Connections
from GA.parentSelector.parentSelector import RandomParentSelector, TournamentSelection, LinealOrder, WheelSelection
from GA.geneticAlgorithm import GenerationalGA, TwoLevelGA

In [ ]:
# Chromosome parameters
mutation_rate = 0.1
mutation_rate_1 = mutation_rate * 2
ChromosomeFilterGrow.grow_prob = mutation_rate_1
ChromosomeFilterGrow.decrease_prob = mutation_rate_1
ChromosomeFilterGrow.skip_prob = 0.1

CNNLayer_WOMP.k_prob = mutation_rate
CNNLayer_WOMP.filter_prob = mutation_rate_1
CNNLayer_WOMP.act_prob = mutation_rate

NNLayer.units_prob = mutation_rate_1
NNLayer.act_prob = mutation_rate
NNLayer.drop_prob = mutation_rate_1

ChromosomeFilterGrow.num_blocks = 3
ChromosomeFilterGrow.initial_filters = 32

'''
data_folder = '../../datasets'
command = 'python3 ./train_gen.py'
verbose = 0
experiments_folder = '../../exps'
gpus = 2
'''
data_folder = '/home/daniel/datasets/MNIST_variations'
command = 'python ./../train_gen.py'
verbose = 1
experiments_folder = '../../exps'
gpus = 1


In [ ]:
experiments_folder = experiments_folder
os.makedirs(experiments_folder, exist_ok=True)

fitness_cnn = FitnessCNN()

In [ ]:
l = CNNLayer_WOMP(1.0, 3, 'relu')
l2 = CNNLayer_WOMP(1.2, 3, 'relu')
l3 = NNLayer(128, 'relu', 0.5)
cnn_layers = [l, l2]
c = ChromosomeFilterGrow(cnn_layers, [l3], Connections.random_connections(len(cnn_layers)))
d = c.random_individual()

In [ ]:
print(d)

In [ ]:
# m = d.decode(input_shape=(28,28,1), num_classes=10, verb=True)

In [ ]:
# dataset params:
data_folder = data_folder
classes = []

# genetic algorithm params:
N = 5
generations = N
population_first_level = N
population_second_level = int(0.6 * N)
training_hours = 100
save_progress = True
maximize_fitness = False
statistical_validation = False
frequency_second_level = 1
start_level2 = 1


# Fitness params
epochs = 20
batch_size = 128
verbose = verbose
redu_plat = False
early_stop = 0
warm_up_epochs= 0
base_lr = 0.02
smooth = 0.1
cosine_dec = False
lr_find = False
precise_eps = 60

datasets = ['MB', 'MBI', 'MRB', 'MRD', 'MRDBI']
datasets = ['cifar10']

In [ ]:
for dataset in datasets:
    print("\nEVOLVING IN DATASET %s ...\n" % dataset)
    exp_folder = os.path.join(experiments_folder, dataset)
    folder = os.path.join(exp_folder, 'genetic')
    fitness_folder = exp_folder
    fitness_file = os.path.join(fitness_folder, 'fitness_example')   
    os.makedirs(folder, exist_ok=True)

    # Load data
    dm = DataManager(dataset, clases=classes, folder_var_mnist=data_folder)
    data = dm.load_data()
    fitness_cnn.set_params(data=data, verbose=verbose, batch_size=batch_size, reduce_plateau=redu_plat,
                   epochs=epochs, cosine_decay=cosine_dec, early_stop=early_stop, 
                   warm_epochs=warm_up_epochs, base_lr=base_lr, smooth_label=smooth, find_lr=lr_find,
                   precise_epochs=precise_eps)

    fitness_cnn.save(fitness_file)

    del dm, data
    
    fitness = FitnessCNNParallel()
    fitness.set_params(chrom_files_folder=fitness_folder, fitness_file=fitness_file, max_gpus=gpus,
                   fp=32, main_line=command)
    generational = TwoLevelGA(chromosome=c,
                              fitness=fitness,
                              generations=generations,
                              population_first_level=population_first_level,
                              population_second_level=population_second_level,
                              training_hours=training_hours,
                              save_progress=save_progress,
                              maximize_fitness=maximize_fitness,
                              statistical_validation=statistical_validation,
                              folder=folder,
                              start_level2=start_level2,
                              frequency_second_level=frequency_second_level)
        
    ti_all = time()
    winner, best_fit, ranking = generational.evolve()
    print("Total elapsed time: %0.3f" % (time() - ti_all))

In [ ]:
ranking1 = generational.evaluate_population(level=1)
ranking2 = generational.evaluate_population(level=2)

In [ ]:
ranking1

In [ ]:
ranking2

In [ ]:
w = generational.best_individual['winner']

In [ ]:
print(generational.history_fitness[w.__repr__()])
print(generational.history_precision_fitness[w.__repr__()])

In [ ]:
generational.best_individual

In [ ]:
for k,f in generational.history_precision_fitness.items():
    print(k,f, end="\n"*2)